## Extract a JSON sidecar template from a BIDS dataset.

The steps are:

1. Set the dataset location (`bids_root_path`) to the absolute path to the root of your BIDS dataset.
2. Set the columns to be excluded (`column_to_skip`) from the summary. Usually, these are columns
that are specific or unique to each event including `onset`, `duration`, `sample`, and `HED`.
The skip columns will not be annotated.
3. Create a dictionary of the unique values in each column.
4. Output the results.

The example below uses a small version of the Wakeman-Hanson face-processing dataset
available on openNeuro as ds003654.

In [1]:
import os
import json
from hed.tools import ColumnDict, hed_to_df
from hed.util import get_file_list


bids_root_path =  os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')),
                                               os.path.join('../../datasets/eeg_ds003654s_hed')))
event_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events")
column_to_skip = ["onset", "duration", "sample", "stim_file", "trial", "response_time"]
col_dict = ColumnDict(skip_cols=column_to_skip, name=bids_root_path)
col_dict.update(event_files)
sidecar_template = col_dict.extract_sidecar_template()
str_json = json.dumps(sidecar_template, indent=4)
print(str_json)
spreadsheet = hed_to_df(sidecar_template)

Extracting template.....
column_name:event_type
column_valuedouble_press
column_valueleft_press
column_valueright_press
column_valuesetup_right_sym
column_valueshow_circle
column_valueshow_cross
column_valueshow_face
column_valueshow_face_initial
column_name:face_type
column_valuefamous_face
column_valuen/a
column_valuescrambled_face
column_valueunfamiliar_face
column_name:rep_status
column_valuedelayed_repeat
column_valuefirst_show
column_valueimmediate_repeat
column_valuen/a
column_name:rep_lag
column_value1
column_value10
column_value11
column_value12
column_value13
column_value14
column_value15
column_value6
column_value7
column_value8
column_value9
column_valuen/a
column_name:value
column_value0
column_value1
column_value13
column_value14
column_value15
column_value17
column_value18
column_value19
column_value256
column_value3
column_value4096
column_value4352
column_value5
column_value6
column_value7
{
    "event_type": {
        "Description": "Description for event_type",
     